KWIKVAL - Rui Cruzeiro - Dezembro 2022

O objectivo deste documento é a criação de uma ferramenta simples de avaliação de apartamentos (Kwikval) com recurso a Machine Learning. Os passos dados pela ferramenta são os seguintes:
1. INPUTS: Pedir ao utilizador um distrito, um concelho, coordenadas geográficas e a Área Bruta Privativa do imóvel (conforme registado na Caderneta Predial);
2. API: Pesquisar os anúncios do Idealista que mais se assemelham ao fornecido pelo utilizador. Para tal, foi criada uma conta no RapidAPI que permite 500 pesquisas gratuitas por mês na API do Idealista;
3. MÉTODO TRADICIONAL (COMPARATIVO DE MERCADO): Fazer homogeneização de áreas e dedução da margem de negociação;
4. MACHINE LEARNING: Aplicação de Regressão Linear com o Scikit-Learn (o que corresponde à aplicação do Método Comparativo de Mercado (método tradicional de avaliação imobiliária).

In [249]:
import numpy as np
import pandas as pd
import requests
from math import radians, cos, sin, asin, sqrt
from concelhos import dados_concelhos
from idealista_api import api_key
from sklearn.linear_model import LinearRegression

## INPUTS

O utilizador introduz um distrito, um concelho, a Área Bruta Privativa e a Área Bruta Dependente. Os três primeiros serão usados para construir o request a fazer à API do Idealista.

In [217]:
# Exemplo de trabalho

user_distrito = 'Coimbra'
user_concelho = 'Coimbra'
user_area = 83
user_dep = 39

O utilizador fornece também as coordenadas de localização do imóvel. Estas serão usadas para seleccionar os imóveis mais próximos entre todos aqueles devolvidos pelo request à API.

In [218]:
user_lat = 40.201609
user_lon = -8.398287
user_coord_str = str(user_lat) + ',' + str(user_lon)

As coordenadas fornecidas pelo utilizador não podem ultrapassar os limites geográficos de Portugal.

In [177]:
# Pontos geográficos extremos de Portugal
max_lat_N = 42.153978
max_lat_S = 32.632960
max_lon_W = -31.274995
max_lon_E = -6.189363

## API

Para obter uma lista de imóveis à venda numa zona, é necessário fazer um GET request à API introduzindo o ID e o nome da zona. O ID da zona corresponde à concatenação apresentada no campo seguinte, em que o dicionário `dados_concelhos`, construído manualmente, faz corresponder um número ao concelho inserido pelo utilizador. Na maior parte dos concelhos, esse número corresponde ao código da Autoridade Tributária para o concelho. O nome da zona é obtido na própria API partindo do ID.

Obtenção do ID da zona:

In [219]:
zona_id = '0-EU-PT-' + dados_concelhos[user_distrito][user_concelho]
zona_id

'0-EU-PT-06-03'

Obtenção do nome da zona:

In [220]:
# Obtém json da API com informação sobre IDs e nomes de zonas similares ao do concelho fornecido pelo utilizador

url = "https://idealista2.p.rapidapi.com/auto-complete"
querystring = {"prefix":user_concelho, "country":"pt"}

headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
zonas = response.json()

# Compara as zonas obtidas acima com a zona pretendida através do ID da zona e obtém o nome

for zona in zonas['locations']:
    try:
        if zona['locationId'] == zona_id:
            zona_nome = zona['name']
    except Exception:
        pass
    
zona_nome

'Coimbra, Coimbra'

Com o ID e o nome, é possível obter a lista das propriedades no concelho escolhido (usando também as coordenadas). Esta informação é actualizada diariamente pelo Idealista. A API só permite um número máximo de 40 resultados por request, portanto são feitos 3 requests às 3 primeiras páginas de resultados correspondentes.
É feito um pedido de anúncios activos de imóveis residenciais com uma variação de área de 10% relativamente ao imóvel em avaliação.

In [221]:
# Recolhe a informação nas 3 primeiras páginas de resultados do Idealista
# (O Idealista não lança mais do que 40 resultados por página, mesmo que maxItens seja superior)

url = "https://idealista2.p.rapidapi.com/properties/list"

resultados_pesq = []

for i in range(1,4): # 3 requests
    
    querystring = {
        "locationId": zona_id,
        "locationName": zona_nome,
        "operation": "sale",
        "numPage": str(i), # 3 primeiras páginas
        "propertyType": "homes",
        "center": user_coord_str,
        "distance": 10,
        "maxSize": int(user_area * 1.1),
        "minSize": int(user_area * 0.9),
        "state": "active",
        "maxItems": 50,
        "sort": "asc", # ascendente em preço
        "locale": "pt",
        "country": "pt"
    }

    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "idealista2.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    resultados_pesq.append(response.json()['elementList'])

# Cria uma lista "flattened" com os 120 resultados    
resultados_lista = [item for sublist in resultados_pesq for item in sublist]

A API só permite filtrar por `"homes"`, o que inclui todo o tipo de imóveis residenciais, não só apartamentos. Portanto, é necessário filtrar a lista obtida acima por `"flat"`.

Nota: Os resultados filtrados encontrados a seguir serão usados na secção de Machine Learning.

In [239]:
# Filtra apenas os apartamentos

resultados_filtrados = []

for resultado in resultados_lista:
    if resultado['propertyType'] == 'flat':
        resultados_filtrados.append(resultado)

## MÉTODO TRADICIONAL (COMPARATIVO DE MERCADO)

De todos os resultados, são seleccionados os 10 com coordenadas mais próximas à fornecida pelo utilizador. Para cálculo da distância entre o imóvel em avaliação e os da lista, foi usada a fórmula de haversine. Esta foi definida na função seguinte.

In [223]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Distância entre dois pontos de uma esfera a partir das suas latitudes e longitudes
    """
    
    # Converte graus decimais para radianos
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Fórmula de haversine 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Raio da Terra em quilómetros
    return c * r

In [224]:
# Calcula as distâncias haversine e devolve a lista dos 10 imóveis mais próximos (comparáveis para o Método Comparativo)

distancias_lista = []

for resultado in resultados_filtrados:
    distancias_lista.append(haversine(user_lat, user_lon, float(resultado['latitude']), float(resultado['longitude'])))
    
distancias_ind = [distancias_lista.index(dist) for dist in sorted(distancias_lista)[:10]]

comparaveis = [resultados_filtrados[ind] for ind in distancias_ind]

Estes são os 10 apartamentos mais próximos geograficamente do apartamento que se pretende avaliar:

In [225]:
for comparavel in comparaveis:
    print('https://www.idealista.pt/imovel/' + comparavel['propertyCode'])

https://www.idealista.pt/imovel/32245918
https://www.idealista.pt/imovel/32245918
https://www.idealista.pt/imovel/32196614
https://www.idealista.pt/imovel/32256241
https://www.idealista.pt/imovel/31585637
https://www.idealista.pt/imovel/32237085
https://www.idealista.pt/imovel/32331860
https://www.idealista.pt/imovel/32187569
https://www.idealista.pt/imovel/32340979
https://www.idealista.pt/imovel/31426856


Visto que o valor do apartamento não varia linearmente com a área, é necessário fazer homogeneização de áreas. Esta foi feita com base na Fórmula de Ajustamento de Pulsares do Prof. Ruy Figueiredo, que ajusta a área de imóveis residenciais calculando um factor de homogeneização pela fórmula seguinte:

$f_{homogeneização} = \Biggl( \dfrac{A_{comparável}}{A_{imóvel \thinspace em \thinspace avaliação}}\ \Biggr) ^{1/4} $

Na homogeneização, é também deduzida uma percentagem de 5% ao valor anunciado dos comparáveis, correspondente à habitual margem de negociação.

In [226]:
val_unit_comparaveis = [comparavel['price']/comparavel['size'] for comparavel in comparaveis]
f_homog_areas = [(comparavel['size']/user_area)**0.25 for comparavel in comparaveis]
f_margem_neg = [0.95 for comparavel in comparaveis]

A multiplicação destas três listas foi feita transformando-as em arrays do NumPy.

In [243]:
val_unit_homogeneizado = np.array(val_unit_comparaveis) * np.array(f_homog_areas) * np.array(f_margem_neg)

Assim, pode finalmente calcular-se o valor do imóvel pelo Método Comparativo de Mercado (um método tradicional de avaliação). É habitual considerar-se para a Área Bruta Dependente um valor unitário correspondente a 30% daquele que é encontrado para a Área Bruta Privativa. Aqui fez-se o mesmo.

In [247]:
val_unit_imovel = sum(aaa) / len(aaa)
valor_area = val_unit_imovel * user_area
valor_dep = val_unit_imovel * user_dep * 0.3
valor_tradicional = valor_area + valor_dep
print('O valor do apartamento obtido pelo Método Comparativo de Mercado é ' + str(int(round(valor_tradicional,-2))) + ' €.')
print('(arredondado às centenas)')

O valor do apartamento obtido pelo Método Comparativo de Mercado é 204300 €.
(arredondado às centenas)


## MACHINE LEARNING

De seguida, procura determinar-se o valor do imóvel através de Machine Learning. Usando a Regressão Linear do Scikit-Learn, vamos treinar o modelo com os resultados devolvidos pela API (apenas os apartamentos obtidos após a filtragem). Seria desejável uma quantidade de dados superior, mas os requests gratuitos à API são limitados. De qualquer modo, tendo sido feita uma filtragem prévia dos apartamentos por localização e área, e sendo o número de features reduzido, considera-se que a amostra usada é coesa. No fundo, trata-se de uma Regressão Linear mais completa do que a que é feita manualmente pelo Método Comparativo de Mercado.

O passo inicial é visualizar os dados e perceber que informações fornecidas pela API podem ser relevantes na determinação do valor do apartamento. Para isso, foi criada uma Dataframe com os resultados filtrados.

In [263]:
df = pd.DataFrame(resultados_filtrados)
df.head()

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,preferenceHighlight,topHighlight,superTopHighlight,topNewDevelopment,district,priceDropValue,dropDate,priceDropPercentage,features,highlightComment
0,31839009,https://img3.idealista.pt/blur/WEB_LISTING-M/0...,GLTS2203,22,1,68000.0,flat,sale,82.0,False,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,31426856,https://img3.idealista.pt/blur/WEB_LISTING-M/0...,125431005-112,17,st,100000.0,flat,sale,81.0,False,...,False,False,False,False,Solum,NaN,NaN,NaN,NaN,NaN
2,32256874,https://img3.idealista.pt/blur/WEB_LISTING-M/0...,C0242-02010,6,bj,111000.0,flat,sale,88.0,False,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,32255879,https://img3.idealista.pt/blur/WEB_LISTING-M/0...,CBRIP 4603,21,bj,116000.0,flat,sale,84.0,False,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,32254699,https://img3.idealista.pt/blur/WEB_LISTING-M/0...,123351024-397,21,4,116000.0,flat,sale,84.0,False,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN


A API devolve muitos campos que são desnecessários para a nossa análise. De seguida, extraem-se apenas os que nos interessam.

Nota: A informação sobre se uma propriedade é nova ou não está, muitas vezes, por preencher nos anúncios. Desconheço o motivo, mas pela minha experiência na consulta do Idealista para fazer avaliações, acho preferível ignorar os campos `newDevelopment` e `newProperty`.

In [264]:
df = df[['floor', 'price', 'size', 'rooms', 'bathrooms', 'status', 'hasLift', 'parkingSpace']]
df.head()

,floor,price,size,rooms,bathrooms,status,hasLift,parkingSpace
0,1,68000.0,82.0,2,1,good,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
1,st,100000.0,81.0,3,1,good,False,NaN
2,bj,111000.0,88.0,4,1,renew,False,NaN
3,bj,116000.0,84.0,2,1,good,False,NaN
4,4,116000.0,84.0,2,1,good,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."


Próximos passos:
    incluir o numero de quartos no metodo tradicional
    fazer o encoding dos campos floor, rooms, bathroomes, status, hasLift e parkingSpace